## Spark Streaming 을 위한 데이터 준비

로컬 Docker 환경 내 Kafka Broker 와 Zookeeper 를 가정합니다.

https://github.com/1ambda/practical-data-pipeline-code 레포지토리 루트에서 아래 커맨드를 이용할 수 있습니다.

```bash
make compose.clean compose.storage-all
```

만약 Kafka Python Library 가 깔려있지 않을 경우 아래와 같이 Pyenv 환경 내에서 설치할 수 있습니다.

```bash
pyenv activate {PYENV_NAME} # 본인이 생성한 Pyenv 이름 설정
pip install kafka-python==2.0.2
pip install msgpack==1.0.3
```

In [4]:
from kafka import KafkaProducer
from kafka import KafkaConsumer
from kafka.client_async import KafkaClient
from kafka.admin import KafkaAdminClient, NewTopic
from kafka import TopicPartition

import json
import msgpack

In [11]:
zookeeper_servers = ["127.0.0.1:2181"]
bootstrap_servers = ["127.0.0.1:9092"]

### Topic 생성

Kafka Partition 을 생성합니다. 일반적으로는 Kafka 관리자가 요청을 받아 UI 나 Console 에서 직접 생성할 수 있습니다.  
API 를 이용해 생성하는 경우는 많지 않으나, 이 노트북에선 실습을 위해 API 를 통해 생성하는 예제를 사용합니다.

In [12]:
admin = KafkaAdminClient(bootstrap_servers=bootstrap_servers)

In [13]:
# partition = 3, replication = 1 설정을 가진 토픽을 생성합니다.
topicAirbnbListing = "airbnb-listing-test"
admin.create_topics([NewTopic(topicAirbnbListing, 3, 1)])

TopicAlreadyExistsError: [Error 36] TopicAlreadyExistsError: Request 'CreateTopicsRequest_v3(create_topic_requests=[(topic='airbnb-listing-test', num_partitions=3, replication_factor=1, replica_assignment=[], configs=[])], timeout=30000, validate_only=False)' failed with response 'CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='airbnb-listing-test', error_code=36, error_message="Topic 'airbnb-listing-test' already exists.")])'.

In [14]:
admin.list_topics()

['_schemas', 'airbnb-listing-test', '__consumer_offsets']

### Kafka Producer 를 이용해 데이터 전송

kafka-python API 를 이용해 데이터를 Broker 로 전송합니다.  
일반적으로는 Java / Scala 언어로 이루어진 개발한 Producer 나 Kafka Connect 등을 이용합니다.  

다만 이 노트북에선 실습으로 Python API 를 이용하기 위해 Pandas 로 CSV 파일을 읽어 Python Kafka API 로 전송합니다.

In [15]:
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)

In [18]:
DATASET_ROOT = "/Users/hoon.park/github/1ambda/practical-data-pipeline-code"
DATASET_AIRBNB_LISTING = f"{DATASET_ROOT}/_datasets/airbnb/airbnb_listings.csv"

In [19]:
import pandas as pd
pdfAirbnbListnig = pd.read_csv(DATASET_AIRBNB_LISTING)

In [20]:
pdfAirbnbListnig.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,360,https://www.airbnb.com/rooms/360,20191129210509,2019-11-30,LoHi Secret garden at the Chickadee Cottage,Come enjoy our oasis is the city and stay at o...,Chickadee Cottage is the largest of our guest ...,Come enjoy our oasis is the city and stay at o...,none,those who are interested in our local brews - ...,...,t,f,moderate,t,t,2,2,0,0,6.00
1,590,https://www.airbnb.com/rooms/590,20191129210509,2019-11-30,Comfortable - and a great value!,"Large guest room in my home, where I also live...",I have been enjoying welcoming many wonderful ...,"Large guest room in my home, where I also live...",none,I love the diversity of my neighborhood and it...,...,f,f,flexible,f,f,2,0,2,0,4.53


In [21]:
# Pandas Row 는 Tuple 입니다. 이를 Dict 로 바꾸기 위해 to_dict 함수를 호출해
# 전달 받은 Row (Dict) 를 Kafka Producer 를 이용해 Topic 으로 보냅니다.

for row in pdfAirbnbListnig.head(2).to_dict(orient="records"):
    producer.send(topicAirbnbListing, bytes(str(row), 'utf-8'))

### Console Consumer 로 데이터 읽기


이제 Broker 에 접근해 Console Consumer 로 해당 토픽 데이터를 읽어봅니다. 컨테이너에 들어가 다음 커맨드를 실행할 수 있습니다.  

```
# 터미널에서 도커로 로그인하기 위해 `docker exec -it broker /bin/bash`

kafka-console-consumer --bootstrap-server broker:9092 --topic airbnb-listing-test --from-beginning
```